In [6]:
from collections import Counter
from flask import Flask, request, render_template, jsonify
import pickle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import argparse
import time
from splinter import Browser
import time
from collections import defaultdict
import json
import re

In [22]:
import uuid

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.cleaner import Cleaner 
import src.fantasy_scoring as fs

In [4]:
import requests
import boto3
from boto3.dynamodb.conditions import Key, Attr
import json
from decimal import Decimal

In [97]:
#session = requests.Session()
session = boto3.Session(profile_name='personal')

right = old

In [17]:
session.profile_name

'personal'

In [5]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1',aws_access_key_id = '',
aws_secret_access_key = '')
table = dynamodb.Table('nba_bbref')

In [14]:
table.

dynamodb.Table(name='test')

In [98]:
dynamodb = session.client('dynamodb', region_name='us-east-1')

In [24]:
%timeit [uuid.uuid4() for i in range(100)]

330 µs ± 6.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
data = pd.read_csv("data/bb_reference/2016to2019Data_merged.csv", encoding = 'latin-1')

In [26]:
data = data.drop(['Unnamed: 0_y','Unnamed: 0'], axis = 1)
data = data.rename(columns={'Unnamed: 0_x':'gamescore_rank'})

In [29]:
clnr = Cleaner()
data = clnr.transform(data)

In [83]:
data['date']=data['dt'].apply(lambda x: x.strftime("%Y-%m-%d"))

In [86]:
lag['id']= [uuid.uuid4() for i in range(len(lag))]
lag.head()

,slug,date,minutes_06,dk_score_06,dk_per_min_06,dk_no_fg_06,dk_no_fg_per_min_06,assists_06,assists_per_min_06,blocks_06,...,doubles_06,fg_pct_06,ft_pct_06,two_pct_06,three_pct_06,game_score_06,attempted_field_goals_06,attempted_three_point_field_goals_06,w_06,id
0,walljo01,2016-01-01,38.966667,51.0000,1.308811,27.000000,0.692900,13.0,0.333618,1.000000,...,2.000000,0.526316,1.000,0.625000,0.000000,22.100000,19.000000,3.000000,1.000000,ddc308d0-324f-42ea-ac51-8a2de7ab2f8d
1,walljo01,2016-01-03,36.500000,40.0000,1.080458,20.750000,0.559476,9.0,0.240267,0.500000,...,1.500000,0.406015,0.750,0.479167,0.083333,13.950000,20.000000,4.500000,0.500000,552f4103-0a0e-4bb2-9085-9a803b85375a
2,walljo01,2016-01-06,37.794444,41.2500,1.081428,21.083333,0.552514,10.0,0.259229,0.333333,...,1.666667,0.411028,0.500,0.408333,0.388889,13.166667,19.666667,4.333333,0.333333,395192ac-b3df-4435-b711-5436e9e28d49
3,walljo01,2016-01-08,37.487500,39.1875,1.036686,18.562500,0.489590,8.5,0.221769,1.000000,...,1.500000,0.408271,0.625,0.406250,0.391667,11.950000,19.750000,4.500000,0.250000,abad0b4f-8bde-4293-99d9-d0fd276d8ec4
4,walljo01,2016-01-09,37.023333,41.9000,1.129349,20.300000,0.546649,8.8,0.234287,0.800000,...,1.600000,0.444264,0.700,0.452273,0.413333,13.720000,19.200000,4.800000,0.400000,88a8eb9b-b97f-4961-9574-e75a49a2af86


In [99]:
table = dynamodb.Table('nba_bbref')  


AttributeError: 'DynamoDB' object has no attribute 'Table'

In [84]:
dynamodb.batch_write_item()

In [104]:
lag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 24 columns):
slug                                    217 non-null object
date                                    217 non-null object
minutes_06                              217 non-null object
dk_score_06                             217 non-null object
dk_per_min_06                           217 non-null object
dk_no_fg_06                             217 non-null object
dk_no_fg_per_min_06                     217 non-null object
assists_06                              217 non-null object
assists_per_min_06                      217 non-null object
blocks_06                               217 non-null object
blocks_per_min_06                       217 non-null object
or_per_min_06                           217 non-null object
dr_per_min_06                           217 non-null object
steals_per_min_06                       217 non-null object
turnovers_per_min_06                    217 non-null ob

In [178]:
single = data[data['player']=='John Wall']
lag = clnr.gameplay_stats(single,player_col = 'slug',lag = 10)
lag['id']= lag['date']+lag['slug']
for ind, elem in enumerate(lag.iloc[0,:]):
    if (str(type(elem))=="<class 'numpy.float64'>"):
        lag.iloc[:,ind] = lag.iloc[:,ind].apply(lambda x : Decimal(x).quantize(Decimal('.001')))

In [162]:
with table.batch_writer() as batch:
    for row in lag.iterrows():
        batch.put_item(Item = row[1].to_dict())

In [181]:
dict_row = lag.iloc[0,:].to_dict()
key  = dict_row.pop('id')
ean = {"#"+key : key for key in dict_row.keys()}
eav = {":"+key : dict_row[key] for key in dict_row.keys()}
ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])
with table.batch_writer() as batch:
    for row in lag.iterrows():
        dict_row = row[1].to_dict()
        key  = dict_row.pop('id')
        #ean = {"#"+key : key for key in dict_row.keys()}
        eav = {":"+key : dict_row[key] for key in dict_row.keys()}
        #ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])
        table.update_item(Key={'id':key},
                 ExpressionAttributeNames = ean,
                 ExpressionAttributeValues = eav,
                 UpdateExpression = ue)

In [180]:
dict_row = lag.iloc[0,:].to_dict()
key  = dict_row.pop('id')
table.update_item(Key={'id':key},
                 ExpressionAttributeNames = ean,
                 ExpressionAttributeValues = eav,
                 UpdateExpression = ue)

ClientError: An error occurred (ValidationException) when calling the UpdateItem operation: Invalid UpdateExpression: An expression attribute value used in expression is not defined; attribute value: :w_06

In [172]:
ean = {"#"+key : key for key in dict_row.keys()}
eav = {":"+key : dict_row[key] for key in dict_row.keys()}
ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])

In [175]:
key

'2016-01-01walljo01'

In [124]:
#lag['minutes_06'].apply(lambda x: Decimal(x))
pe = "slug, #dt, minutes_06"
fe = Key('date').lt('2016-11-07')
ea = {"#dt":"date"}
table.scan(ProjectionExpression = pe,ExpressionAttributeNames =ea, FilterExpression = fe)

{'Items': [{'slug': 'walljo01',
   'date': '2016-01-28',
   'minutes_06': Decimal('36.753')},
  {'slug': 'walljo01', 'date': '2016-10-27', 'minutes_06': Decimal('35.158')},
  {'slug': 'walljo01', 'date': '2016-02-06', 'minutes_06': Decimal('37.508')},
  {'slug': 'walljo01', 'date': '2016-01-11', 'minutes_06': Decimal('36.642')},
  {'slug': 'walljo01', 'date': '2016-10-30', 'minutes_06': Decimal('35.708')},
  {'slug': 'walljo01', 'date': '2016-02-19', 'minutes_06': Decimal('37.758')},
  {'slug': 'walljo01', 'date': '2016-01-26', 'minutes_06': Decimal('35.156')},
  {'slug': 'walljo01', 'date': '2016-02-29', 'minutes_06': Decimal('34.475')},
  {'slug': 'walljo01', 'date': '2016-03-12', 'minutes_06': Decimal('36.697')},
  {'slug': 'walljo01', 'date': '2016-01-08', 'minutes_06': Decimal('37.487')},
  {'slug': 'walljo01', 'date': '2016-01-01', 'minutes_06': Decimal('38.967')},
  {'slug': 'walljo01', 'date': '2016-01-09', 'minutes_06': Decimal('37.023')},
  {'slug': 'walljo01', 'date': '2016-

In [160]:
table.query(IndexName = 'slug-date-index', KeyConditionExpression = Key('slug').eq('walljo01') & Key('date').lt('2016-01-20'),
            ExpressionAttributeNames =ea,
            ProjectionExpression = pe,
            ScanIndexForward = False,
            Limit = 1
    )['Items'][0]

{'slug': 'walljo01', 'date': '2016-01-18', 'minutes_06': Decimal('36.369')}

In [141]:
table.get_item(IndexName = 'slug-date-index', KeyConditionExpression = Key('slug').eq('walljo01') & Key('date').lt('2016-01-20'),
            ExpressionAttributeNames =ea,
            ProjectionExpression = pe,
            ScanIndexForward = False,
               
    )

ParamValidationError: Parameter validation failed:
Missing required parameter in input: "Key"
Unknown parameter in input: "IndexName", must be one of: TableName, Key, AttributesToGet, ConsistentRead, ReturnConsumedCapacity, ProjectionExpression, ExpressionAttributeNames
Unknown parameter in input: "KeyConditionExpression", must be one of: TableName, Key, AttributesToGet, ConsistentRead, ReturnConsumedCapacity, ProjectionExpression, ExpressionAttributeNames
Unknown parameter in input: "ScanIndexForward", must be one of: TableName, Key, AttributesToGet, ConsistentRead, ReturnConsumedCapacity, ProjectionExpression, ExpressionAttributeNames

In [82]:
lag.head()

,slug,date,minutes_06,dk_score_06,dk_per_min_06,dk_no_fg_06,dk_no_fg_per_min_06,assists_06,assists_per_min_06,blocks_06,...,turnovers_per_min_06,doubles_06,fg_pct_06,ft_pct_06,two_pct_06,three_pct_06,game_score_06,attempted_field_goals_06,attempted_three_point_field_goals_06,w_06
0,walljo01,1/1/2016,38.966667,51.000000,1.308811,27.000000,0.692900,13.000000,0.333618,1.0,...,0.128315,2.0,0.526316,1.000000,0.625000,0.000000,22.100000,19.000000,3.000000,1.000000
1,walljo01,1/10/2017,39.566667,54.500000,1.376397,29.250000,0.738566,13.500000,0.341083,1.0,...,0.126398,2.0,0.525063,0.875000,0.575658,0.250000,22.450000,20.000000,2.500000,1.000000
2,walljo01,1/10/2018,40.488889,58.333333,1.437283,29.333333,0.724661,12.666667,0.314003,1.0,...,0.147258,2.0,0.522881,0.773810,0.558375,0.333333,23.666667,22.333333,3.666667,0.666667
3,walljo01,1/11/2016,39.050000,55.687500,1.421652,29.437500,0.757627,12.000000,0.307479,1.0,...,0.124839,2.0,0.484266,0.830357,0.502115,0.375000,21.625000,21.500000,3.750000,0.750000
4,walljo01,1/11/2017,38.983333,52.900000,1.352991,30.100000,0.775280,11.800000,0.302806,1.0,...,0.105037,1.8,0.425508,0.764286,0.451692,0.300000,18.980000,21.400000,4.000000,0.600000


In [80]:
data.sort_values(by = 'date')

,gamescore_rank,assists,attempted_field_goals,attempted_free_throws,attempted_three_point_field_goals,blocks,date,defensive_rebounds,game_score,location,...,turnovers_per_min,points_per_min,double_per_min,minutes,fg_pct,w,three_pct,two_pct,ft_pct,dt
0,0,13,19,4,3,1,1/1/2016,1,22.1,Location.HOME,...,0.128315,0.615911,0.051326,38.966667,0.526316,1,0.000000,0.625000,1.000000,2016-01-01
12068,49,0,4,2,1,0,1/1/2016,1,6.7,Location.AWAY,...,0.000000,0.779221,0.000000,10.266667,0.750000,0,0.000000,1.000000,1.000000,2016-01-01
12139,50,0,5,2,2,0,1/1/2016,1,6.7,Location.AWAY,...,0.000000,0.471050,0.000000,16.983333,0.600000,0,0.000000,1.000000,1.000000,2016-01-01
12227,51,2,8,0,0,2,1/1/2016,3,6.7,Location.AWAY,...,0.198020,0.660066,0.000000,15.150000,0.625000,0,0.000000,0.625000,0.000000,2016-01-01
12417,52,1,11,0,7,0,1/1/2016,2,6.5,Location.HOME,...,0.037879,0.303030,0.151515,26.400000,0.272727,1,0.285714,0.250000,0.000000,2016-01-01
4792,19,2,9,2,3,0,1/1/2016,8,12.7,Location.HOME,...,0.100503,0.536013,0.000000,29.850000,0.666667,1,0.666667,0.666667,1.000000,2016-01-01
12606,53,1,7,2,3,0,1/1/2016,4,6.4,Location.HOME,...,0.000000,0.304912,0.000000,29.516667,0.428571,1,0.333333,0.500000,1.000000,2016-01-01
12748,54,5,4,2,1,0,1/1/2016,2,6.4,Location.AWAY,...,0.000000,0.182371,0.000000,21.933333,0.250000,0,0.000000,0.333333,1.000000,2016-01-01
13014,55,0,5,0,0,1,1/1/2016,1,6.4,Location.AWAY,...,0.000000,0.632689,0.105448,9.483333,0.600000,0,0.000000,0.600000,0.000000,2016-01-01
13283,56,0,3,0,1,0,1/1/2016,5,6.2,Location.HOME,...,0.000000,0.229183,0.045837,21.816667,0.666667,1,1.000000,0.500000,0.000000,2016-01-01


In [ ]:
alpha = 0.3
n = 561
f2 = 'dk_score'
combos = [(('dk_per_k','median'),('dk_per_k','mean')),
          (('dk_per_k','median'),('dk_score','median')),
          (('dk_per_k','median'),('salary','mean')),
          (('salary','mean'),('dk_per_k','mean')),
          #('salary','dk_per_k'),
          #('minutes','dk_per_min'),
          #('salary','dk_score'),
          #('salary','minutes')
          #('salary','dk_per_min'),
          #('dk_score','dk_per_min')
         ]
fig, ax = plt.subplots(len(combos),1,figsize=(24,36))
buckets = [get_n_scatter(summary,elem[0],elem[1],ax = ax[ind],alpha= alpha,n = n, s = 44) for ind,elem in enumerate(combos)]
#buckets = get_n_scatter(data,f1,f2,ax = ax[0], alpha , n )
plt.show()